<a href="https://colab.research.google.com/github/Badm0nting/stable-diffusion/blob/main/stable_diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **SETUP**

In [ ]:
%pip install --quiet --upgrade diffusers transformers accelerate mediapy omegaconf

In [ ]:
!pip install omegaconf

In [ ]:
use_refiner = False

## **MODEL**

In [ ]:
#@title Setup Model
# @markdown IF U WANT TO USE ANOTHER MODEL, GO TO [HERE](https://huggingface.co/models?pipeline_tag=text-to-image&sort=trending), MAKE SURE THE MODEL HAVE A SAFETENSORS FILE

import mediapy as media
import random
import sys
import torch

from diffusers import DiffusionPipeline

model = "digiplay/Realisian_v5" # @param ["digiplay/Realisian_v5", "stabilityai/stable-diffusion-xl-base-1.0", "hakurei/waifu-diffusion", "Linaqruf/anything-v3.0"]

pipe = DiffusionPipeline.from_pretrained(
    model,
    torch_dtype=torch.float16,
    safety_checker=None,
    requires_safety_checker=False
)

pipe = pipe.to("cuda")  # If you have a GPU for acceleration

##**PROMPT**##

In [ ]:
# @title PROMPT { display-mode: "form" }
import os
prompt = "lebron james" #@param {type:"string"}
seed = 3442633626 # @param {type:"slider", min:0, max:9000000000, step:1}

negative_prompt = "bad-picture-chill-75v, ng_deepnegative_v1_75t, badhandv4, (worst quality:2), (low quality:2), (normal quality:2), (lowres:2), (bad anatomy:2), (bad hands:2), (watermark:2), (mole:1.5), (freckles:1.5)" #@param {type:"string"}

width = 512  #@param {type:"slider", min:8, max:2048, step:8}
height = 720  #@param {type:"slider", min:8, max:2048, step:8}

width = int(width)
height = int(height)

images = pipe(
    prompt=prompt,
    width=width,
    height=height,
    negative_prompt=negative_prompt,
    output_type="latent" if use_refiner else "pil",
    generator=torch.Generator().manual_seed(seed)
).images
if use_refiner:
  images = refiner(
      prompt = prompt,
      negative_prompt = negative_prompt,
      image = images,
      ).images

print(f"Prompt:\t{prompt}\nSeed:\t{seed}")
base_filename = "output.jpg"
new_filename = base_filename
if os.path.exists(base_filename):
    index = 1
    while True:
        new_filename = f"output_{index}.jpg"
        if not os.path.exists(new_filename):
            break
        index += 1
images[0].save(new_filename)
media.show_images(images)

## **TOOLS**

In [ ]:
#@title FIX FALSE NSFW
pipe.safety_checker = None
pipe.requires_safety_checker = False